In [26]:
# import all the libraries needed for this project
import numpy as np
import pandas as pd
import math
from math import sqrt
from numpy import mean
from numpy.random import rand
from numpy.random import randn
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense , Activation, Dropout, CategoryEncoding
from keras.optimizers import Adam ,RMSprop
from keras import  backend as K
#from tensorflow.keras.utils import to_categorical

In [61]:
train=pd.read_csv("/content/train.csv")
test_x=pd.read_csv("/content/test.csv")

In [62]:
from sklearn.model_selection import train_test_split

#test_x=test_x.drop(["Id"],axis=1)
#train=train.drop(["Id"],axis=1)
Y=train["SalePrice"]
df=train.drop(["SalePrice"],axis=1)
df=pd.concat([train, test_x])


In [63]:
df

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500.0
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500.0
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500.0
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000.0
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,2915,160,RM,21.0,1936,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,6,2006,WD,Normal,NaN
1455,2916,160,RM,21.0,1894,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2006,WD,Abnorml,NaN
1456,2917,20,RL,160.0,20000,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2006,WD,Abnorml,NaN
1457,2918,85,RL,62.0,10441,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,Shed,700,7,2006,WD,Normal,NaN


In [64]:
#getting list of columns with Null Values again
print(round(df.isnull().sum()/len(df.index)*100,2).sort_values(ascending=False)[ round(df.isnull().sum()/len(df.index),2) > 0 ] )

PoolQC          99.66
MiscFeature     96.40
Alley           93.22
Fence           80.44
MasVnrType      60.50
SalePrice       49.98
FireplaceQu     48.65
LotFrontage     16.65
GarageYrBlt      5.45
GarageFinish     5.45
GarageQual       5.45
GarageCond       5.45
GarageType       5.38
BsmtExposure     2.81
BsmtCond         2.81
BsmtQual         2.77
BsmtFinType2     2.74
BsmtFinType1     2.71
MasVnrArea       0.79
dtype: float64


In [65]:
df['age']=df['YrSold']-df['YearBuilt']
df['garage_age'] = df['YrSold'] - df['GarageYrBlt']
df['remodel_age'] = df['YrSold'] - df['YearRemodAdd']

df.drop('GarageYrBlt',axis=1,inplace=True)



In [66]:
#From the Data Description
#NA = No Pool for PoolQC
df['PoolQC'] = df['PoolQC'].fillna('None')
#NA = No Misc Feature for MiscFeature
df['MiscFeature'] = df['MiscFeature'].fillna('None')
#NA = No Alley
df['Alley'] = df['Alley'].fillna('None')
#NA = No Fireplace for FireplaceQu
df['FireplaceQu'] = df['FireplaceQu'].fillna('None')
#NA = No Fence
df['Fence'] = df['Fence'].fillna('None')
#NA = No Fireplace
df['FireplaceQu'] = df['FireplaceQu'].fillna('None')
#NA = No Garage
df['GarageCond'] = df['GarageCond'].fillna('None')
df['GarageType'] = df['GarageType'].fillna('None')
df['GarageFinish'] = df['GarageFinish'].fillna('None')
df['GarageQual'] = df['GarageQual'].fillna('None')
df['garage_age'] = df['GarageQual'].fillna(0)   #No Garage No Age
#NA = No Basement
df['BsmtExposure'] = df['BsmtExposure'].fillna('None')
df['BsmtFinType2'] = df['BsmtFinType2'].fillna('None')
df['BsmtCond'] = df['BsmtCond'].fillna('None')
df['BsmtQual'] = df['BsmtQual'].fillna('None')
df['BsmtFinType1'] = df['BsmtFinType1'].fillna('None')
#NA = No Masonery Area
df['MasVnrType'] = df['MasVnrType'].fillna('None')
df['MasVnrArea'] = df['MasVnrArea'].fillna(0)

In [67]:
#LotFrontage : Replacing Null value with the median of the neighbourhood
df['LotFrontage'] = df.groupby('Neighborhood')['LotFrontage'].transform(lambda x: x.fillna(x.median()))
# Filling the Electrical 0.06% rows with the mode
df['Electrical'] = df['Electrical'].fillna(df['Electrical'].mode()[0])

In [68]:
df

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,age,garage_age,remodel_age
0,1,60,RL,65.0,8450,Pave,None,Reg,Lvl,AllPub,...,None,0,2,2008,WD,Normal,208500.0,5,TA,5
1,2,20,RL,80.0,9600,Pave,None,Reg,Lvl,AllPub,...,None,0,5,2007,WD,Normal,181500.0,31,TA,31
2,3,60,RL,68.0,11250,Pave,None,IR1,Lvl,AllPub,...,None,0,9,2008,WD,Normal,223500.0,7,TA,6
3,4,70,RL,60.0,9550,Pave,None,IR1,Lvl,AllPub,...,None,0,2,2006,WD,Abnorml,140000.0,91,TA,36
4,5,60,RL,84.0,14260,Pave,None,IR1,Lvl,AllPub,...,None,0,12,2008,WD,Normal,250000.0,8,TA,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,2915,160,RM,21.0,1936,Pave,None,Reg,Lvl,AllPub,...,None,0,6,2006,WD,Normal,NaN,36,None,36
1455,2916,160,RM,21.0,1894,Pave,None,Reg,Lvl,AllPub,...,None,0,4,2006,WD,Abnorml,NaN,36,TA,36
1456,2917,20,RL,160.0,20000,Pave,None,Reg,Lvl,AllPub,...,None,0,9,2006,WD,Abnorml,NaN,46,TA,10
1457,2918,85,RL,62.0,10441,Pave,None,Reg,Lvl,AllPub,...,Shed,700,7,2006,WD,Normal,NaN,14,None,14


In [69]:
#obtain all the numerical features
ids = df["Id"]
df=df.drop(["Id"],axis=1)
numerical_features = df.dtypes[df.dtypes != 'object'].index
non_numerical_features= df.dtypes[df.dtypes=="object"].index
#train_x[non_numerical_features]=to_categorical(train_x[non_numerical_features])
#apply standardization to each feature
df=df.fillna(0)
df[numerical_features] = df[numerical_features].apply( lambda x: (x-x.mean()) / x.std())
df = pd.get_dummies(df, columns=non_numerical_features, dtype=float)



In [70]:
df["id"] = ids
sam_test = df[df["id"] > 1460]
train_x = df[df["id"] <= 1460]




In [71]:
sam_test = sam_test.drop(["id"],axis=1)
train_x = train_x.drop(["id"],axis=1)
train_x

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial,garage_age_Ex,garage_age_Fa,garage_age_Gd,garage_age_None,garage_age_Po,garage_age_TA
0,0.067320,-0.209323,-0.217841,0.646073,-0.507197,1.046078,0.896679,0.528944,0.581046,-0.292975,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,-0.873466,0.481817,-0.072032,-0.063174,2.187904,0.154737,-0.395536,-0.566919,1.178053,-0.292975,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.067320,-0.071095,0.137173,0.646073,-0.507197,0.980053,0.848819,0.338845,0.098173,-0.292975,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.302516,-0.439703,-0.078371,0.646073,-0.507197,-1.859033,-0.682695,-0.566919,-0.494445,-0.292975,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.067320,0.666122,0.518814,1.355319,-0.507197,0.947040,0.753100,1.389978,0.469107,-0.292975,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,0.067320,-0.347551,-0.285421,-0.063174,-0.507197,0.914028,0.753100,-0.566919,-0.968539,-0.292975,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1456,-0.873466,0.712198,0.381246,-0.063174,0.391170,0.220763,0.178782,0.098426,0.765416,0.670501,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1457,0.302516,-0.163247,-0.142781,0.646073,3.086271,-1.000704,1.040259,-0.566919,-0.364947,-0.292975,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1458,-0.873466,-0.071095,-0.057197,-0.772420,0.391170,-0.703591,0.561660,-0.566919,-0.860990,5.789336,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [72]:
X_train, X_test, y_train, y_test = train_test_split(train_x, Y, test_size=0.15, random_state=42)

In [73]:
y_train

984     126000
777     142500
707     254000
598     217500
874      66500
         ...  
1095    176432
1130    135000
1294    115000
860     189950
1126    174000
Name: SalePrice, Length: 1241, dtype: int64

In [74]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential,layers
from keras.layers import BatchNormalization

In [17]:
#Min max for test and train data
y_min=np.min(y_train)
y_max=np.max(y_train)
y_train=(y_train- y_min)/ (y_max - y_min)

y_min=np.min(y_test)
y_max=np.max(y_test)
y_test=(y_test- y_min)/ (y_max - y_min)

In [75]:
y_train

984     126000
777     142500
707     254000
598     217500
874      66500
         ...  
1095    176432
1130    135000
1294    115000
860     189950
1126    174000
Name: SalePrice, Length: 1241, dtype: int64

In [95]:
from keras.layers import Input
model = Sequential(
    [
        Dense(units=400, activation='linear', input_dim=317),
        BatchNormalization(),
        Dropout(0.05),
        Dense(500, name="layer2"),
        tf.keras.layers.LeakyReLU(alpha=0.2),
        BatchNormalization(),
        Dropout(0.05),
        Dense(units = 1, activation = "linear")  # Output layer with linear activation
    ]
)


In [96]:
from keras.losses import BinaryCrossentropy
model.compile(loss="mean_squared_error",
                  optimizer="adam",
                  metrics=["mean_squared_error"])


In [76]:
xtrain, xval, ytrain, yval = train_test_split(X_train, y_train, test_size=0.15, random_state=42)

In [98]:
model.fit(xtrain, ytrain, epochs=50,batch_size = 64,
                  validation_data=(xval, yval))

Epoch 1/50
17/17 [==============================] - 2s 24ms/step - loss: 1.7922 - mean_squared_error: 1.7922 - val_loss: 0.2658 - val_mean_squared_error: 0.2658
Epoch 2/50
17/17 [==============================] - 0s 13ms/step - loss: 1.0614 - mean_squared_error: 1.0614 - val_loss: 0.0936 - val_mean_squared_error: 0.0936
Epoch 3/50
17/17 [==============================] - 0s 12ms/step - loss: 0.5548 - mean_squared_error: 0.5548 - val_loss: 0.0703 - val_mean_squared_error: 0.0703
Epoch 4/50
17/17 [==============================] - 0s 13ms/step - loss: 0.4292 - mean_squared_error: 0.4292 - val_loss: 0.0626 - val_mean_squared_error: 0.0626
Epoch 5/50
17/17 [==============================] - 0s 13ms/step - loss: 0.3209 - mean_squared_error: 0.3209 - val_loss: 0.0578 - val_mean_squared_error: 0.0578
Epoch 6/50
17/17 [==============================] - 0s 12ms/step - loss: 0.2428 - mean_squared_error: 0.2428 - val_loss: 0.0645 - val_mean_squared_error: 0.0645
Epoch 7/50
17/17 [================

In [99]:
model.evaluate(X_test, y_test)

7/7 [==============================] - 0s 4ms/step - loss: 0.0083 - mean_squared_error: 0.0083


[0.00834157969802618, 0.00834157969802618]

In [27]:
model.compile(loss="mean_squared_error",
                  optimizer=RMSprop(learning_rate=0.05),
                  metrics=["mean_squared_error"])

In [28]:
model.fit(xtrain, ytrain, epochs=50,
                  validation_data=(xval, yval))

Epoch 1/50
33/33 [==============================] - 2s 25ms/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 2/50
33/33 [==============================] - 1s 16ms/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 3/50
33/33 [==============================] - 1s 22ms/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 4/50
33/33 [==============================] - 0s 9ms/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 5/50
33/33 [==============================] - 0s 10ms/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 6/50
33/33 [==============================] - 0s 9ms/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 7/50
33/33 [==============================] - 0s 8ms/step - loss: nan - mean_squared_error: nan - val_

,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,...,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial,garage_age_Ex,garage_age_Fa,garage_age_Gd,garage_age_None,garage_age_Po,garage_age_TA
664,-0.943964,1.039865,1.374324,-0.517023,1.116853,1.023678,-0.570555,2.800626,-0.288554,-0.478063,...,-0.091003,-0.117811,-2.137613,3.266908,-0.045361,-0.184312,-0.098363,-0.242277,-0.045361,0.33701
971,-1.523455,-0.826431,0.651256,-0.517023,1.050634,0.926804,0.015951,0.270469,-0.288554,-0.837900,...,-0.091003,-0.117811,0.467491,-0.305890,-0.045361,-0.184312,-0.098363,-0.242277,-0.045361,0.33701
1364,-1.790913,-0.735060,0.651256,-0.517023,1.116853,0.975241,-0.570555,-0.972685,-0.288554,0.074139,...,-0.091003,-0.117811,-2.137613,-0.305890,-0.045361,-0.184312,-0.098363,-0.242277,-0.045361,0.33701
1166,-0.275320,-0.004191,1.374324,-0.517023,1.216181,1.120552,-0.172173,-0.972685,-0.288554,2.549997,...,-0.091003,-0.117811,0.467491,-0.305890,-0.045361,-0.184312,-0.098363,-0.242277,-0.045361,0.33701
991,2.265528,0.716760,1.374324,3.077516,-2.955603,0.054939,-0.570555,-0.499103,-0.288554,0.300451,...,-0.091003,-0.117811,0.467491,-0.305890,-0.045361,-0.184312,-0.098363,-0.242277,-0.045361,0.33701
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1207,-0.007862,-0.138442,-0.071812,-0.517023,1.050634,0.878367,0.093414,-0.227231,-0.288554,1.753378,...,-0.091003,-0.117811,0.467491,-0.305890,-0.045361,-0.184312,-0.098363,-0.242277,-0.045361,0.33701
1351,-0.007862,-0.127221,-0.071812,0.381612,-0.306852,-1.107547,1.188963,-0.273274,-0.288554,-0.063912,...,-0.091003,-0.117811,0.467491,-0.305890,-0.045361,-0.184312,-0.098363,-0.242277,-0.045361,0.33701
40,0.616206,-0.186232,-0.071812,-0.517023,-0.207523,-0.962236,-0.011714,0.437100,-0.288554,-0.276645,...,-0.091003,-0.117811,-2.137613,-0.305890,-0.045361,-0.184312,-0.098363,-0.242277,-0.045361,0.33701
116,0.081291,0.110124,-0.794879,-0.517023,-0.306852,-1.107547,0.071282,-0.599958,3.864701,-0.713428,...,-0.091003,-0.117811,0.467491,-0.305890,-0.045361,-0.184312,-0.098363,-0.242277,-0.045361,0.33701


In [37]:
model.evaluate(X_test, y_test)

7/7 [==============================] - 0s 3ms/step - loss: 7.9791 - mean_squared_error: 7.9791


[7.979106426239014, 7.979106426239014]

In [100]:
model2 = Sequential(
    [
        Dense(units=500, activation='linear', input_dim=317),
        BatchNormalization(),
        Dropout(0.05),
        Dense(400, name="layer2"),
        tf.keras.layers.LeakyReLU(alpha=0.2),
        BatchNormalization(),
        Dropout(0.05),
        Dense(units = 1, activation = "linear")  # Output layer with linear activation
    ]
)

In [101]:
model3 = Sequential(
    [
        Dense(units=600, activation='linear', input_dim=317),
        BatchNormalization(),
        Dropout(0.05),
        Dense(300, name="layer2"),
        tf.keras.layers.LeakyReLU(alpha=0.2),
        BatchNormalization(),
        Dropout(0.05),
        Dense(units = 1, activation = "linear")  # Output layer with linear activation
    ]
)

In [102]:
model4 = Sequential(
    [
        Dense(units=700, activation='linear', input_dim=317),
        BatchNormalization(),
        Dropout(0.05),
        Dense(200, name="layer2"),
        tf.keras.layers.LeakyReLU(alpha=0.2),
        BatchNormalization(),
        Dropout(0.05),
        Dense(units = 1, activation = "linear")  # Output layer with linear activation
    ]
)

In [103]:
model2.compile(loss="mean_squared_error",
                  optimizer="adam",
                  metrics=["mean_squared_error"])
model3.compile(loss="mean_squared_error",
                  optimizer="adam",
                  metrics=["mean_squared_error"])
model4.compile(loss="mean_squared_error",
                  optimizer="adam",
                  metrics=["mean_squared_error"])

In [104]:
model2.fit(xtrain, ytrain, epochs=50,
                  validation_data=(xval, yval))

Epoch 1/50
33/33 [==============================] - 5s 60ms/step - loss: 1.4930 - mean_squared_error: 1.4930 - val_loss: 0.1229 - val_mean_squared_error: 0.1229
Epoch 2/50
33/33 [==============================] - 1s 18ms/step - loss: 0.7506 - mean_squared_error: 0.7506 - val_loss: 0.0675 - val_mean_squared_error: 0.0675
Epoch 3/50
33/33 [==============================] - 1s 18ms/step - loss: 0.3223 - mean_squared_error: 0.3223 - val_loss: 0.1227 - val_mean_squared_error: 0.1227
Epoch 4/50
33/33 [==============================] - 1s 15ms/step - loss: 0.3210 - mean_squared_error: 0.3210 - val_loss: 0.0762 - val_mean_squared_error: 0.0762
Epoch 5/50
33/33 [==============================] - 1s 16ms/step - loss: 0.1892 - mean_squared_error: 0.1892 - val_loss: 0.1816 - val_mean_squared_error: 0.1816
Epoch 6/50
33/33 [==============================] - 1s 17ms/step - loss: 0.1485 - mean_squared_error: 0.1485 - val_loss: 0.0520 - val_mean_squared_error: 0.0520
Epoch 7/50
33/33 [================

In [105]:
model2.evaluate(X_test, y_test)

7/7 [==============================] - 0s 4ms/step - loss: 0.0025 - mean_squared_error: 0.0025


[0.0025311014614999294, 0.0025311014614999294]

In [106]:
model3.fit(xtrain, ytrain, epochs=50,
                  validation_data=(xval, yval ))

Epoch 1/50
33/33 [==============================] - 2s 15ms/step - loss: 1.9808 - mean_squared_error: 1.9808 - val_loss: 0.1335 - val_mean_squared_error: 0.1335
Epoch 2/50
33/33 [==============================] - 0s 10ms/step - loss: 0.6797 - mean_squared_error: 0.6797 - val_loss: 0.1110 - val_mean_squared_error: 0.1110
Epoch 3/50
33/33 [==============================] - 0s 11ms/step - loss: 0.3136 - mean_squared_error: 0.3136 - val_loss: 0.1216 - val_mean_squared_error: 0.1216
Epoch 4/50
33/33 [==============================] - 0s 10ms/step - loss: 0.2227 - mean_squared_error: 0.2227 - val_loss: 0.1530 - val_mean_squared_error: 0.1530
Epoch 5/50
33/33 [==============================] - 0s 10ms/step - loss: 0.1861 - mean_squared_error: 0.1861 - val_loss: 0.0727 - val_mean_squared_error: 0.0727
Epoch 6/50
33/33 [==============================] - 0s 11ms/step - loss: 0.1759 - mean_squared_error: 0.1759 - val_loss: 0.0553 - val_mean_squared_error: 0.0553
Epoch 7/50
33/33 [================

In [107]:
model3.evaluate(X_test, y_test)

7/7 [==============================] - 0s 4ms/step - loss: 0.0019 - mean_squared_error: 0.0019


[0.0018512451788410544, 0.0018512451788410544]

In [108]:
model4.fit(xtrain, ytrain, epochs=50,
                  validation_data=(xval, yval))

Epoch 1/50
33/33 [==============================] - 2s 24ms/step - loss: 1.5167 - mean_squared_error: 1.5167 - val_loss: 0.3406 - val_mean_squared_error: 0.3406
Epoch 2/50
33/33 [==============================] - 1s 16ms/step - loss: 0.5818 - mean_squared_error: 0.5818 - val_loss: 0.0874 - val_mean_squared_error: 0.0874
Epoch 3/50
33/33 [==============================] - 1s 17ms/step - loss: 0.3172 - mean_squared_error: 0.3172 - val_loss: 0.0531 - val_mean_squared_error: 0.0531
Epoch 4/50
33/33 [==============================] - 1s 17ms/step - loss: 0.1840 - mean_squared_error: 0.1840 - val_loss: 0.0675 - val_mean_squared_error: 0.0675
Epoch 5/50
33/33 [==============================] - 0s 11ms/step - loss: 0.1621 - mean_squared_error: 0.1621 - val_loss: 0.0419 - val_mean_squared_error: 0.0419
Epoch 6/50
33/33 [==============================] - 0s 9ms/step - loss: 0.1577 - mean_squared_error: 0.1577 - val_loss: 0.0357 - val_mean_squared_error: 0.0357
Epoch 7/50
33/33 [=================

In [109]:
model4.evaluate(X_test, y_test)

7/7 [==============================] - 0s 4ms/step - loss: 0.0022 - mean_squared_error: 0.0022


[0.0022453053388744593, 0.0022453053388744593]

In [110]:
model5 = Sequential(
    [
        Dense(units=800, activation='linear', input_dim=317),
        BatchNormalization(),
        Dropout(0.05),
        Dense(100, name="layer2"),
        tf.keras.layers.LeakyReLU(alpha=0.2),
        BatchNormalization(),
        Dropout(0.05),
        Dense(units = 1, activation = "linear")  # Output layer with linear activation
    ]
)
model5.compile(loss="mean_squared_error",
                  optimizer="adam",
                  metrics=["mean_squared_error"])
model5.fit(xtrain, ytrain, epochs=50,
                  validation_data=(xval, yval))
model5.evaluate(X_test, y_test)

Epoch 1/50
33/33 [==============================] - 2s 15ms/step - loss: 1.2596 - mean_squared_error: 1.2596 - val_loss: 0.0959 - val_mean_squared_error: 0.0959
Epoch 2/50
33/33 [==============================] - 0s 9ms/step - loss: 0.3240 - mean_squared_error: 0.3240 - val_loss: 0.0550 - val_mean_squared_error: 0.0550
Epoch 3/50
33/33 [==============================] - 0s 10ms/step - loss: 0.2097 - mean_squared_error: 0.2097 - val_loss: 0.0574 - val_mean_squared_error: 0.0574
Epoch 4/50
33/33 [==============================] - 0s 10ms/step - loss: 0.1455 - mean_squared_error: 0.1455 - val_loss: 0.0299 - val_mean_squared_error: 0.0299
Epoch 5/50
33/33 [==============================] - 0s 10ms/step - loss: 0.1480 - mean_squared_error: 0.1480 - val_loss: 0.0251 - val_mean_squared_error: 0.0251
Epoch 6/50
33/33 [==============================] - 0s 10ms/step - loss: 0.1369 - mean_squared_error: 0.1369 - val_loss: 0.0336 - val_mean_squared_error: 0.0336
Epoch 7/50
33/33 [=================

[0.001487859059125185, 0.001487859059125185]

In [111]:
model6 = Sequential(
    [
        Dense(units=900, activation='linear', input_dim=317),
        BatchNormalization(),
        Dropout(0.05),
        Dense(units = 1, activation = "linear")  # Output layer with linear activation
    ]
)
model6.compile(loss="mean_squared_error",
                  optimizer="adam",
                  metrics=["mean_squared_error"])
model6.fit(xtrain, ytrain, epochs=50,
                  validation_data=(xval, yval))
model6.evaluate(X_test, y_test)

Epoch 1/50
33/33 [==============================] - 2s 22ms/step - loss: 1.8649 - mean_squared_error: 1.8649 - val_loss: 0.1232 - val_mean_squared_error: 0.1232
Epoch 2/50
33/33 [==============================] - 0s 14ms/step - loss: 1.0727 - mean_squared_error: 1.0727 - val_loss: 0.7478 - val_mean_squared_error: 0.7478
Epoch 3/50
33/33 [==============================] - 1s 15ms/step - loss: 0.9876 - mean_squared_error: 0.9876 - val_loss: 0.2466 - val_mean_squared_error: 0.2466
Epoch 4/50
33/33 [==============================] - 0s 14ms/step - loss: 0.4612 - mean_squared_error: 0.4612 - val_loss: 0.1607 - val_mean_squared_error: 0.1607
Epoch 5/50
33/33 [==============================] - 0s 9ms/step - loss: 0.2848 - mean_squared_error: 0.2848 - val_loss: 0.1272 - val_mean_squared_error: 0.1272
Epoch 6/50
33/33 [==============================] - 0s 8ms/step - loss: 0.2048 - mean_squared_error: 0.2048 - val_loss: 0.0778 - val_mean_squared_error: 0.0778
Epoch 7/50
33/33 [==================

[0.0053168716840445995, 0.0053168716840445995]

In [112]:
model7 = Sequential(
    [
        Dense(units=900, activation='linear', input_dim=317),
        BatchNormalization(),
        Dropout(0.05),
        Dense(1, name="layer2"),
        tf.keras.layers.LeakyReLU(alpha=0.2),
        BatchNormalization(),
        Dropout(0.05),
        Dense(units = 1, activation = "linear")  # Output layer with linear activation
    ]
)
model7.compile(loss="mean_squared_error",
                  optimizer="adam",
                  metrics=["mean_squared_error"])
model7.fit(xtrain, ytrain, epochs=50,
                  validation_data=(xval, yval))
model7.evaluate(X_test, y_test)

Epoch 1/50
33/33 [==============================] - 2s 15ms/step - loss: 0.5660 - mean_squared_error: 0.5660 - val_loss: 0.4751 - val_mean_squared_error: 0.4751
Epoch 2/50
33/33 [==============================] - 0s 9ms/step - loss: 0.4594 - mean_squared_error: 0.4594 - val_loss: 0.2473 - val_mean_squared_error: 0.2473
Epoch 3/50
33/33 [==============================] - 0s 9ms/step - loss: 0.3799 - mean_squared_error: 0.3799 - val_loss: 0.1933 - val_mean_squared_error: 0.1933
Epoch 4/50
33/33 [==============================] - 0s 9ms/step - loss: 0.3214 - mean_squared_error: 0.3214 - val_loss: 0.2172 - val_mean_squared_error: 0.2172
Epoch 5/50
33/33 [==============================] - 0s 10ms/step - loss: 0.2741 - mean_squared_error: 0.2741 - val_loss: 0.2002 - val_mean_squared_error: 0.2002
Epoch 6/50
33/33 [==============================] - 0s 8ms/step - loss: 0.2289 - mean_squared_error: 0.2289 - val_loss: 0.1684 - val_mean_squared_error: 0.1684
Epoch 7/50
33/33 [====================

[0.0009519907180219889, 0.0009519907180219889]

In [113]:
model8 = Sequential(
    [
        Dense(units=200, activation='linear', input_dim=317),
        BatchNormalization(),
        Dropout(0.05),
        Dense(700, name="layer2"),
        tf.keras.layers.LeakyReLU(alpha=0.2),
        BatchNormalization(),
        Dropout(0.05),
        Dense(units = 1, activation = "linear")  # Output layer with linear activation
    ]
)
model8.compile(loss="mean_squared_error",
                  optimizer="adam",
                  metrics=["mean_squared_error"])
model8.fit(xtrain, ytrain, epochs=50,
                  validation_data=(xval, yval))
model8.evaluate(X_test, y_test)

Epoch 1/50
33/33 [==============================] - 2s 14ms/step - loss: 1.6267 - mean_squared_error: 1.6267 - val_loss: 0.1313 - val_mean_squared_error: 0.1313
Epoch 2/50
33/33 [==============================] - 0s 8ms/step - loss: 0.8856 - mean_squared_error: 0.8856 - val_loss: 0.0760 - val_mean_squared_error: 0.0760
Epoch 3/50
33/33 [==============================] - 0s 9ms/step - loss: 0.5895 - mean_squared_error: 0.5895 - val_loss: 0.1070 - val_mean_squared_error: 0.1070
Epoch 4/50
33/33 [==============================] - 0s 9ms/step - loss: 0.4749 - mean_squared_error: 0.4749 - val_loss: 0.2576 - val_mean_squared_error: 0.2576
Epoch 5/50
33/33 [==============================] - 0s 8ms/step - loss: 0.3653 - mean_squared_error: 0.3653 - val_loss: 0.1047 - val_mean_squared_error: 0.1047
Epoch 6/50
33/33 [==============================] - 0s 8ms/step - loss: 0.2547 - mean_squared_error: 0.2547 - val_loss: 0.0803 - val_mean_squared_error: 0.0803
Epoch 7/50
33/33 [=====================

[0.0030948494095355272, 0.0030948494095355272]

In [114]:
model9 = Sequential(
    [
        Dense(units=450, activation='linear', input_dim=317),
        BatchNormalization(),
        Dropout(0.05),
        Dense(450, name="layer2"),
        tf.keras.layers.LeakyReLU(alpha=0.2),
        BatchNormalization(),
        Dropout(0.05),
        Dense(units = 1, activation = "linear")  # Output layer with linear activation
    ]
)
model9.compile(loss="mean_squared_error",
                  optimizer="adam",
                  metrics=["mean_squared_error"])
model9.fit(xtrain, ytrain, epochs=50,
                  validation_data=(xval, yval))
model9.evaluate(X_test, y_test)

Epoch 1/50
33/33 [==============================] - 3s 26ms/step - loss: 1.6681 - mean_squared_error: 1.6681 - val_loss: 0.1110 - val_mean_squared_error: 0.1110
Epoch 2/50
33/33 [==============================] - 1s 18ms/step - loss: 0.6904 - mean_squared_error: 0.6904 - val_loss: 0.0740 - val_mean_squared_error: 0.0740
Epoch 3/50
33/33 [==============================] - 0s 13ms/step - loss: 0.3089 - mean_squared_error: 0.3089 - val_loss: 0.1099 - val_mean_squared_error: 0.1099
Epoch 4/50
33/33 [==============================] - 0s 11ms/step - loss: 0.2385 - mean_squared_error: 0.2385 - val_loss: 0.0736 - val_mean_squared_error: 0.0736
Epoch 5/50
33/33 [==============================] - 0s 10ms/step - loss: 0.1852 - mean_squared_error: 0.1852 - val_loss: 0.0630 - val_mean_squared_error: 0.0630
Epoch 6/50
33/33 [==============================] - 0s 10ms/step - loss: 0.1806 - mean_squared_error: 0.1806 - val_loss: 0.1144 - val_mean_squared_error: 0.1144
Epoch 7/50
33/33 [================

[0.00241164886392653, 0.00241164886392653]

In [115]:
model10 = Sequential(
    [
        Dense(units=1000, activation='linear', input_dim=317),
        BatchNormalization(),
        Dropout(0.05),
        Dense(200, name="layer2"),
        tf.keras.layers.LeakyReLU(alpha=0.2),
        BatchNormalization(),
        Dropout(0.05),
        Dense(units = 1, activation = "linear")  # Output layer with linear activation
    ]
)
model10.compile(loss="mean_squared_error",
                  optimizer="adam",
                  metrics=["mean_squared_error"])
model10.fit(xtrain, ytrain, epochs=50,
                  validation_data=(xval, yval))
model10.evaluate(X_test, y_test)

Epoch 1/50
33/33 [==============================] - 2s 18ms/step - loss: 2.0572 - mean_squared_error: 2.0572 - val_loss: 0.1563 - val_mean_squared_error: 0.1563
Epoch 2/50
33/33 [==============================] - 0s 12ms/step - loss: 0.4812 - mean_squared_error: 0.4812 - val_loss: 0.0945 - val_mean_squared_error: 0.0945
Epoch 3/50
33/33 [==============================] - 0s 11ms/step - loss: 0.2297 - mean_squared_error: 0.2297 - val_loss: 0.0616 - val_mean_squared_error: 0.0616
Epoch 4/50
33/33 [==============================] - 0s 13ms/step - loss: 0.1907 - mean_squared_error: 0.1907 - val_loss: 0.1079 - val_mean_squared_error: 0.1079
Epoch 5/50
33/33 [==============================] - 0s 11ms/step - loss: 0.1963 - mean_squared_error: 0.1963 - val_loss: 0.0491 - val_mean_squared_error: 0.0491
Epoch 6/50
33/33 [==============================] - 0s 11ms/step - loss: 0.1550 - mean_squared_error: 0.1550 - val_loss: 0.0335 - val_mean_squared_error: 0.0335
Epoch 7/50
33/33 [================

[0.0016557611525058746, 0.0016557611525058746]

In [116]:
model11 = Sequential(
    [
        Dense(units=2000, activation='linear', input_dim=317),
        BatchNormalization(),
        Dropout(0.05),
        Dense(200, name="layer2"),
        tf.keras.layers.LeakyReLU(alpha=0.2),
        BatchNormalization(),
        Dropout(0.05),
        Dense(units = 1, activation = "linear")  # Output layer with linear activation
    ]
)
model11.compile(loss="mean_squared_error",
                  optimizer="adam",
                  metrics=["mean_squared_error"])
model11.fit(xtrain, ytrain, epochs=50,
                  validation_data=(xval, yval))
model11.evaluate(X_test, y_test)

Epoch 1/50
33/33 [==============================] - 2s 26ms/step - loss: 3.3101 - mean_squared_error: 3.3101 - val_loss: 0.0850 - val_mean_squared_error: 0.0850
Epoch 2/50
33/33 [==============================] - 1s 21ms/step - loss: 0.3259 - mean_squared_error: 0.3259 - val_loss: 0.0277 - val_mean_squared_error: 0.0277
Epoch 3/50
33/33 [==============================] - 1s 19ms/step - loss: 0.1787 - mean_squared_error: 0.1787 - val_loss: 0.0314 - val_mean_squared_error: 0.0314
Epoch 4/50
33/33 [==============================] - 1s 20ms/step - loss: 0.1430 - mean_squared_error: 0.1430 - val_loss: 0.0293 - val_mean_squared_error: 0.0293
Epoch 5/50
33/33 [==============================] - 1s 19ms/step - loss: 0.1461 - mean_squared_error: 0.1461 - val_loss: 0.0331 - val_mean_squared_error: 0.0331
Epoch 6/50
33/33 [==============================] - 1s 18ms/step - loss: 0.1300 - mean_squared_error: 0.1300 - val_loss: 0.0328 - val_mean_squared_error: 0.0328
Epoch 7/50
33/33 [================

[0.003268956672400236, 0.003268956672400236]

In [117]:
model11_x = Sequential(
    [
        Dense(units=3000, activation='linear', input_dim=317),
        BatchNormalization(),
        Dropout(0.05),
        Dense(200, name="layer2"),
        tf.keras.layers.LeakyReLU(alpha=0.2),
        BatchNormalization(),
        Dropout(0.05),
        Dense(units = 1, activation = "linear")  # Output layer with linear activation
    ]
)
model11_x.compile(loss="mean_squared_error",
                  optimizer="adam",
                  metrics=["mean_squared_error"])
model11_x.fit(xtrain, ytrain, epochs=50,
                  validation_data=(xval, yval))
model11_x.evaluate(X_test, y_test)

Epoch 1/50
33/33 [==============================] - 4s 53ms/step - loss: 3.4449 - mean_squared_error: 3.4449 - val_loss: 0.5500 - val_mean_squared_error: 0.5500
Epoch 2/50
33/33 [==============================] - 1s 31ms/step - loss: 0.3571 - mean_squared_error: 0.3571 - val_loss: 0.0993 - val_mean_squared_error: 0.0993
Epoch 3/50
33/33 [==============================] - 1s 28ms/step - loss: 0.2149 - mean_squared_error: 0.2149 - val_loss: 0.2062 - val_mean_squared_error: 0.2062
Epoch 4/50
33/33 [==============================] - 1s 25ms/step - loss: 0.1776 - mean_squared_error: 0.1776 - val_loss: 0.2244 - val_mean_squared_error: 0.2244
Epoch 5/50
33/33 [==============================] - 1s 26ms/step - loss: 0.1542 - mean_squared_error: 0.1542 - val_loss: 0.2610 - val_mean_squared_error: 0.2610
Epoch 6/50
33/33 [==============================] - 1s 28ms/step - loss: 0.1460 - mean_squared_error: 0.1460 - val_loss: 0.2316 - val_mean_squared_error: 0.2316
Epoch 7/50
33/33 [================

[0.002797213150188327, 0.002797213150188327]

In [118]:
model11_y = Sequential(
    [
        Dense(units=4000, activation='linear', input_dim=317),
        BatchNormalization(),
        Dropout(0.05),
        Dense(200, name="layer2"),
        tf.keras.layers.LeakyReLU(alpha=0.2),
        BatchNormalization(),
        Dropout(0.05),
        Dense(units = 1, activation = "linear")  # Output layer with linear activation
    ]
)
model11_y.compile(loss="mean_squared_error",
                  optimizer="adam",
                  metrics=["mean_squared_error"])
model11_y.fit(xtrain, ytrain, epochs=50,
                  validation_data=(xval, yval))
model11_y.evaluate(X_test, y_test)

Epoch 1/50
33/33 [==============================] - 3s 41ms/step - loss: 4.8835 - mean_squared_error: 4.8835 - val_loss: 0.1159 - val_mean_squared_error: 0.1159
Epoch 2/50
33/33 [==============================] - 1s 31ms/step - loss: 0.2921 - mean_squared_error: 0.2921 - val_loss: 0.0382 - val_mean_squared_error: 0.0382
Epoch 3/50
33/33 [==============================] - 1s 39ms/step - loss: 0.2130 - mean_squared_error: 0.2130 - val_loss: 0.0331 - val_mean_squared_error: 0.0331
Epoch 4/50
33/33 [==============================] - 2s 52ms/step - loss: 0.1745 - mean_squared_error: 0.1745 - val_loss: 0.0286 - val_mean_squared_error: 0.0286
Epoch 5/50
33/33 [==============================] - 2s 48ms/step - loss: 0.1376 - mean_squared_error: 0.1376 - val_loss: 0.0201 - val_mean_squared_error: 0.0201
Epoch 6/50
33/33 [==============================] - 1s 37ms/step - loss: 0.1192 - mean_squared_error: 0.1192 - val_loss: 0.0273 - val_mean_squared_error: 0.0273
Epoch 7/50
33/33 [================

[0.0038339286111295223, 0.0038339286111295223]

In [119]:
model11 = Sequential(
    [
        Dense(units=1000, activation='relu', input_dim=317),
        BatchNormalization(),
        Dropout(0.1),
        Dense(100, name="layer2"),
        tf.keras.layers.LeakyReLU(alpha=0.2),
        BatchNormalization(),
        Dropout(0.1),
        Dense(units = 1, activation = "linear")  # Output layer with linear activation
    ]
)
model11.compile(loss="mean_squared_error",
                  optimizer="adam",
                  metrics=["mean_squared_error"])
model11.fit(xtrain, ytrain, epochs=50,
                  validation_data=(xval, yval))
model11.evaluate(X_test, y_test)

Epoch 1/50
33/33 [==============================] - 2s 17ms/step - loss: 1.3212 - mean_squared_error: 1.3212 - val_loss: 0.1660 - val_mean_squared_error: 0.1660
Epoch 2/50
33/33 [==============================] - 0s 10ms/step - loss: 0.6443 - mean_squared_error: 0.6443 - val_loss: 0.0769 - val_mean_squared_error: 0.0769
Epoch 3/50
33/33 [==============================] - 0s 11ms/step - loss: 0.3107 - mean_squared_error: 0.3107 - val_loss: 0.1030 - val_mean_squared_error: 0.1030
Epoch 4/50
33/33 [==============================] - 0s 10ms/step - loss: 0.2197 - mean_squared_error: 0.2197 - val_loss: 0.0596 - val_mean_squared_error: 0.0596
Epoch 5/50
33/33 [==============================] - 0s 15ms/step - loss: 0.1624 - mean_squared_error: 0.1624 - val_loss: 0.0576 - val_mean_squared_error: 0.0576
Epoch 6/50
33/33 [==============================] - 1s 15ms/step - loss: 0.1744 - mean_squared_error: 0.1744 - val_loss: 0.0585 - val_mean_squared_error: 0.0585
Epoch 7/50
33/33 [================

[0.001489745918661356, 0.001489745918661356]

In [120]:
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

#I asked from AI's to improve my best model, did they make my model better or not ??
#more complex chatgpt's model
model12 = Sequential([
    Dense(units=1024, activation='linear', input_dim=317),
    BatchNormalization(),
    LeakyReLU(alpha=0.2),
    Dropout(0.1),

    Dense(units=512, activation='linear'),
    BatchNormalization(),
    LeakyReLU(alpha=0.2),
    Dropout(0.1),

    Dense(units=256, activation='linear'),
    BatchNormalization(),
    LeakyReLU(alpha=0.2),
    Dropout(0.1),

    Dense(units=128, activation='linear'),
    BatchNormalization(),
    LeakyReLU(alpha=0.2),
    Dropout(0.1),

    Dense(units=1, activation='linear')
])

model12.compile(loss="mean_squared_error",
              optimizer="adam",
              metrics=["mean_squared_error"])

early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.0001)

history = model12.fit(xtrain, ytrain, epochs=50,
                    validation_data=(xval, yval),
                    callbacks=[early_stopping, reduce_lr])

model12.evaluate(X_test, y_test)

Epoch 1/50
33/33 [==============================] - 4s 31ms/step - loss: 0.4461 - mean_squared_error: 0.4461 - val_loss: 0.0884 - val_mean_squared_error: 0.0884 - lr: 0.0010
Epoch 2/50
33/33 [==============================] - 1s 21ms/step - loss: 0.1798 - mean_squared_error: 0.1798 - val_loss: 0.0610 - val_mean_squared_error: 0.0610 - lr: 0.0010
Epoch 3/50
33/33 [==============================] - 1s 23ms/step - loss: 0.1408 - mean_squared_error: 0.1408 - val_loss: 0.0156 - val_mean_squared_error: 0.0156 - lr: 0.0010
Epoch 4/50
33/33 [==============================] - 1s 23ms/step - loss: 0.1188 - mean_squared_error: 0.1188 - val_loss: 0.0565 - val_mean_squared_error: 0.0565 - lr: 0.0010
Epoch 5/50
33/33 [==============================] - 1s 23ms/step - loss: 0.0921 - mean_squared_error: 0.0921 - val_loss: 0.0125 - val_mean_squared_error: 0.0125 - lr: 0.0010
Epoch 6/50
33/33 [==============================] - 1s 22ms/step - loss: 0.0802 - mean_squared_error: 0.0802 - val_loss: 0.0159 - 

[0.0013417882146313787, 0.0013417882146313787]

In [121]:
#gemini's model

model13 = Sequential([
    Dense(units=1024, activation='relu', input_dim=317),  # ReLU for hidden layers
    BatchNormalization(),
    Dropout(0.2),  # Increased dropout rate

    Dense(units=512, activation='relu'),
    BatchNormalization(),
    Dropout(0.2),

    Dense(units=256, activation='relu'),
    BatchNormalization(),
    Dropout(0.2),

    Dense(units=128, activation='relu'),
    BatchNormalization(),
    Dropout(0.2),

    Dense(units=1, activation='linear')  # Linear activation for final prediction
])

model13.compile(loss="mse",  # Mean Squared Error for regression
              optimizer="adam",
              metrics=["mse"])

early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=0.0001)  # Adjusted factor

history = model13.fit(xtrain, ytrain, epochs=50,
                    validation_data=(xval, yval),
                    callbacks=[early_stopping, reduce_lr])

model13.evaluate(X_test, y_test)

Epoch 1/50
33/33 [==============================] - 3s 30ms/step - loss: 1.2418 - mse: 1.2418 - val_loss: 0.0498 - val_mse: 0.0498 - lr: 0.0010
Epoch 2/50
33/33 [==============================] - 1s 23ms/step - loss: 0.7042 - mse: 0.7042 - val_loss: 0.0317 - val_mse: 0.0317 - lr: 0.0010
Epoch 3/50
33/33 [==============================] - 1s 24ms/step - loss: 0.4563 - mse: 0.4563 - val_loss: 0.0325 - val_mse: 0.0325 - lr: 0.0010
Epoch 4/50
33/33 [==============================] - 1s 23ms/step - loss: 0.4302 - mse: 0.4302 - val_loss: 0.0202 - val_mse: 0.0202 - lr: 0.0010
Epoch 5/50
33/33 [==============================] - 1s 22ms/step - loss: 0.3514 - mse: 0.3514 - val_loss: 0.0204 - val_mse: 0.0204 - lr: 0.0010
Epoch 6/50
33/33 [==============================] - 1s 23ms/step - loss: 0.2773 - mse: 0.2773 - val_loss: 0.0292 - val_mse: 0.0292 - lr: 0.0010
Epoch 7/50
33/33 [==============================] - 1s 31ms/step - loss: 0.2450 - mse: 0.2450 - val_loss: 0.0283 - val_mse: 0.0283 - lr:

[0.0016950876452028751, 0.0016950876452028751]

In [122]:
#blackbox's model

model14 = Sequential([
    Dense(units=1024, activation='relu', input_dim=317),
    BatchNormalization(),
    Dropout(0.2),

    Dense(units=768, activation='relu'),
    BatchNormalization(),
    Dropout(0.2),

    Dense(units=512, activation='relu'),
    BatchNormalization(),
    Dropout(0.2),

    Dense(units=256, activation='relu'),
    BatchNormalization(),
    Dropout(0.2),

    Dense(units=1, activation='linear')
])

model14.compile(loss="mean_squared_error",
              optimizer="adam",
              metrics=["mean_squared_error"])

early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.0001)

history = model14.fit(xtrain, ytrain, epochs=50,
                    validation_data=(xval, yval),
                    callbacks=[early_stopping, reduce_lr])

model14.evaluate(X_test, y_test)

Epoch 1/50
33/33 [==============================] - 4s 40ms/step - loss: 1.8700 - mean_squared_error: 1.8700 - val_loss: 0.0297 - val_mean_squared_error: 0.0297 - lr: 0.0010
Epoch 2/50
33/33 [==============================] - 1s 32ms/step - loss: 0.9547 - mean_squared_error: 0.9547 - val_loss: 0.0374 - val_mean_squared_error: 0.0374 - lr: 0.0010
Epoch 3/50
33/33 [==============================] - 1s 32ms/step - loss: 0.6016 - mean_squared_error: 0.6016 - val_loss: 0.0449 - val_mean_squared_error: 0.0449 - lr: 0.0010
Epoch 4/50
33/33 [==============================] - 1s 37ms/step - loss: 0.4745 - mean_squared_error: 0.4745 - val_loss: 0.1026 - val_mean_squared_error: 0.1026 - lr: 0.0010
Epoch 5/50
33/33 [==============================] - 1s 45ms/step - loss: 0.3928 - mean_squared_error: 0.3928 - val_loss: 0.0521 - val_mean_squared_error: 0.0521 - lr: 0.0010
Epoch 6/50
33/33 [==============================] - 2s 47ms/step - loss: 0.3279 - mean_squared_error: 0.3279 - val_loss: 0.0284 - 

[0.0015722505049780011, 0.0015722505049780011]

In [123]:
#copilot's model

model15 = Sequential([
    Dense(units=1024, activation='linear', input_dim=317),
    BatchNormalization(),
    LeakyReLU(alpha=0.2),
    Dropout(0.1),

    Dense(units=512, activation='linear'),
    BatchNormalization(),
    LeakyReLU(alpha=0.2),
    Dropout(0.1),

    Dense(units=256, activation='linear'),
    BatchNormalization(),
    LeakyReLU(alpha=0.2),
    Dropout(0.1),

    Dense(units=128, activation='linear'),
    BatchNormalization(),
    LeakyReLU(alpha=0.2),
    Dropout(0.1),

    Dense(units=1, activation='linear')
])

# Modeli derle
model15.compile(loss="mean_squared_error",
              optimizer="adam",
              metrics=["mean_squared_error"])

# Early stopping ve learning rate düşürme callback'leri ekle
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.0001)

# Veriyle modeli eğit
history = model15.fit(xtrain, ytrain, epochs=50,
                    validation_data=(xval, yval),
                    callbacks=[early_stopping, reduce_lr])

# Test verisi üzerinde modeli değerlendir
model15.evaluate(X_test, y_test)

Epoch 1/50
33/33 [==============================] - 5s 35ms/step - loss: 0.4913 - mean_squared_error: 0.4913 - val_loss: 0.0401 - val_mean_squared_error: 0.0401 - lr: 0.0010
Epoch 2/50
33/33 [==============================] - 1s 24ms/step - loss: 0.1821 - mean_squared_error: 0.1821 - val_loss: 0.0191 - val_mean_squared_error: 0.0191 - lr: 0.0010
Epoch 3/50
33/33 [==============================] - 1s 23ms/step - loss: 0.1335 - mean_squared_error: 0.1335 - val_loss: 0.0155 - val_mean_squared_error: 0.0155 - lr: 0.0010
Epoch 4/50
33/33 [==============================] - 1s 22ms/step - loss: 0.1100 - mean_squared_error: 0.1100 - val_loss: 0.0214 - val_mean_squared_error: 0.0214 - lr: 0.0010
Epoch 5/50
33/33 [==============================] - 1s 23ms/step - loss: 0.0880 - mean_squared_error: 0.0880 - val_loss: 0.0209 - val_mean_squared_error: 0.0209 - lr: 0.0010
Epoch 6/50
33/33 [==============================] - 1s 23ms/step - loss: 0.0854 - mean_squared_error: 0.0854 - val_loss: 0.0125 - 

[0.0020673510152846575, 0.0020673510152846575]

In [124]:
# Now I asked from AI's to use maximum 1000 neurons and make their own models, they can choose their own learning rate, number of layers and optimizer
# Which one is the best ??

In [125]:
#ChatGPT

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, LeakyReLU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Model tanımı
model16 = Sequential([
    Dense(units=300, activation='linear', input_dim=317),
    BatchNormalization(),
    LeakyReLU(alpha=0.2),
    Dropout(0.1),

    Dense(units=250, activation='linear'),
    BatchNormalization(),
    LeakyReLU(alpha=0.2),
    Dropout(0.1),

    Dense(units=200, activation='linear'),
    BatchNormalization(),
    LeakyReLU(alpha=0.2),
    Dropout(0.1),

    Dense(units=150, activation='linear'),
    BatchNormalization(),
    LeakyReLU(alpha=0.2),
    Dropout(0.1),

    Dense(units=100, activation='linear'),
    BatchNormalization(),
    LeakyReLU(alpha=0.2),
    Dropout(0.1),

    Dense(units=1, activation='linear')  # Çıktı katmanı
])

# Model derleme
optimizer = Adam(learning_rate=0.001)
model16.compile(loss="mean_squared_error",
              optimizer=optimizer,
              metrics=["mean_squared_error"])

# Callbacks tanımı
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.0001)

# Model eğitimi
history = model16.fit(xtrain, ytrain, epochs=50,
                    validation_data=(xval, yval),
                    callbacks=[early_stopping, reduce_lr])

# Model değerlendirmesi
model16.evaluate(X_test, y_test)


Epoch 1/50
33/33 [==============================] - 4s 20ms/step - loss: 0.3660 - mean_squared_error: 0.3660 - val_loss: 0.0252 - val_mean_squared_error: 0.0252 - lr: 0.0010
Epoch 2/50
33/33 [==============================] - 0s 10ms/step - loss: 0.1712 - mean_squared_error: 0.1712 - val_loss: 0.0331 - val_mean_squared_error: 0.0331 - lr: 0.0010
Epoch 3/50
33/33 [==============================] - 0s 10ms/step - loss: 0.1650 - mean_squared_error: 0.1650 - val_loss: 0.0220 - val_mean_squared_error: 0.0220 - lr: 0.0010
Epoch 4/50
33/33 [==============================] - 0s 11ms/step - loss: 0.1260 - mean_squared_error: 0.1260 - val_loss: 0.0268 - val_mean_squared_error: 0.0268 - lr: 0.0010
Epoch 5/50
33/33 [==============================] - 0s 11ms/step - loss: 0.0850 - mean_squared_error: 0.0850 - val_loss: 0.0216 - val_mean_squared_error: 0.0216 - lr: 0.0010
Epoch 6/50
33/33 [==============================] - 0s 12ms/step - loss: 0.0843 - mean_squared_error: 0.0843 - val_loss: 0.0174 - 

[0.0008658331935293972, 0.0008658331935293972]

In [127]:
# Geminis Models, it developed 2 model, Idk why?

model17 = Sequential([
    Dense(units=200, activation='relu', input_dim=317),  # 200 nöronlu giriş katmanı
    BatchNormalization(),
    Dropout(0.2),

    Dense(units=200, activation='relu'),  # 200 nöronlu 1. gizli katman
    BatchNormalization(),
    Dropout(0.2),

    Dense(units=200, activation='relu'),  # 200 nöronlu 2. gizli katman
    BatchNormalization(),
    Dropout(0.2),

    Dense(units=200, activation='relu'),  # 200 nöronlu 3. gizli katman
    BatchNormalization(),
    Dropout(0.2),

    Dense(units=1, activation='linear')  # 1 nöronlu çıktı katmanı (fiyat tahmini)
])

model17.compile(loss="mse", optimizer="adam", metrics=["mse"])
model17.fit(xtrain, ytrain, epochs=50,
                  validation_data=(xval, yval))
model17.evaluate(X_test, y_test)




model18 = Sequential([
    Dense(units=250, activation='relu', input_dim=317),  # 250 nöronlu giriş katmanı
    BatchNormalization(),
    Dropout(0.2),

    Dense(units=350, activation='relu'),  # 350 nöronlu 1. gizli katman
    BatchNormalization(),
    Dropout(0.2),

    Dense(units=200, activation='relu'),  # 200 nöronlu 2. gizli katman
    BatchNormalization(),
    Dropout(0.2),

    Dense(units=1, activation='linear')  # 1 nöronlu çıktı katmanı (fiyat tahmini)
])

model18.compile(loss="mse", optimizer="adam", metrics=["mse"])
model18.fit(xtrain, ytrain, epochs=50,
                  validation_data=(xval, yval))
model18.evaluate(X_test, y_test)

Epoch 1/50
33/33 [==============================] - 4s 15ms/step - loss: 1.5994 - mse: 1.5994 - val_loss: 0.0336 - val_mse: 0.0336
Epoch 2/50
33/33 [==============================] - 0s 8ms/step - loss: 1.1347 - mse: 1.1347 - val_loss: 0.0625 - val_mse: 0.0625
Epoch 3/50
33/33 [==============================] - 0s 7ms/step - loss: 0.7628 - mse: 0.7628 - val_loss: 0.0599 - val_mse: 0.0599
Epoch 4/50
33/33 [==============================] - 0s 8ms/step - loss: 0.6431 - mse: 0.6431 - val_loss: 0.0503 - val_mse: 0.0503
Epoch 5/50
33/33 [==============================] - 0s 8ms/step - loss: 0.5570 - mse: 0.5570 - val_loss: 0.0707 - val_mse: 0.0707
Epoch 6/50
33/33 [==============================] - 0s 8ms/step - loss: 0.4633 - mse: 0.4633 - val_loss: 0.0550 - val_mse: 0.0550
Epoch 7/50
33/33 [==============================] - 0s 8ms/step - loss: 0.3371 - mse: 0.3371 - val_loss: 0.0379 - val_mse: 0.0379
Epoch 8/50
33/33 [==============================] - 0s 8ms/step - loss: 0.2808 - mse: 0.2

[0.00128674297593534, 0.00128674297593534]

In [128]:
#copilot

model19 = Sequential([
    Dense(units=512, activation='relu', input_dim=317),
    BatchNormalization(),
    Dropout(0.2),

    Dense(units=256, activation='relu'),
    BatchNormalization(),
    Dropout(0.2),

    Dense(units=128, activation='relu'),
    BatchNormalization(),
    Dropout(0.2),

    Dense(units=64, activation='relu'),
    BatchNormalization(),
    Dropout(0.2),

    Dense(units=1, activation='linear')
])

# Modeli derle
model19.compile(loss="mean_squared_error",
              optimizer="adam",
              metrics=["mean_squared_error"])

# Early stopping ve learning rate düşürme callback'leri ekle
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.0001)

# Veriyle modeli eğit
history = model19.fit(xtrain, ytrain, epochs=50,
                    validation_data=(xval, yval),
                    callbacks=[early_stopping, reduce_lr])

# Test verisi üzerinde modeli değerlendir
model19.evaluate(X_test, y_test)

Epoch 1/50
33/33 [==============================] - 3s 19ms/step - loss: 1.5870 - mean_squared_error: 1.5870 - val_loss: 0.0184 - val_mean_squared_error: 0.0184 - lr: 0.0010
Epoch 2/50
33/33 [==============================] - 0s 11ms/step - loss: 0.8807 - mean_squared_error: 0.8807 - val_loss: 0.0328 - val_mean_squared_error: 0.0328 - lr: 0.0010
Epoch 3/50
33/33 [==============================] - 0s 10ms/step - loss: 0.6241 - mean_squared_error: 0.6241 - val_loss: 0.0336 - val_mean_squared_error: 0.0336 - lr: 0.0010
Epoch 4/50
33/33 [==============================] - 0s 11ms/step - loss: 0.5111 - mean_squared_error: 0.5111 - val_loss: 0.0393 - val_mean_squared_error: 0.0393 - lr: 0.0010
Epoch 5/50
33/33 [==============================] - 0s 10ms/step - loss: 0.4059 - mean_squared_error: 0.4059 - val_loss: 0.0260 - val_mean_squared_error: 0.0260 - lr: 0.0010
Epoch 6/50
33/33 [==============================] - 0s 11ms/step - loss: 0.3395 - mean_squared_error: 0.3395 - val_loss: 0.0193 - 

[0.017142757773399353, 0.017142757773399353]

In [129]:
#blackbox


model20 = Sequential([
    Dense(units=400, activation='relu', input_dim=317),
    BatchNormalization(),
    Dropout(0.2),

    Dense(units=300, activation='relu'),
    BatchNormalization(),
    Dropout(0.2),

    Dense(units=200, activation='relu'),
    BatchNormalization(),
    Dropout(0.2),

    Dense(units=100, activation='linear')
])

model20.compile(loss="mean_squared_error",
              optimizer="rmsprop",
              metrics=["mean_squared_error"])

early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.0001)

history = model20.fit(xtrain, ytrain, epochs=50,
                    validation_data=(xval, yval),
                    callbacks=[early_stopping, reduce_lr])

model20.evaluate(X_test, y_test)


Epoch 1/50
33/33 [==============================] - 2s 21ms/step - loss: 1.0210 - mean_squared_error: 1.0210 - val_loss: 0.0595 - val_mean_squared_error: 0.0595 - lr: 0.0010
Epoch 2/50
33/33 [==============================] - 1s 16ms/step - loss: 0.4475 - mean_squared_error: 0.4475 - val_loss: 0.0464 - val_mean_squared_error: 0.0464 - lr: 0.0010
Epoch 3/50
33/33 [==============================] - 1s 18ms/step - loss: 0.2675 - mean_squared_error: 0.2675 - val_loss: 0.0392 - val_mean_squared_error: 0.0392 - lr: 0.0010
Epoch 4/50
33/33 [==============================] - 1s 18ms/step - loss: 0.1952 - mean_squared_error: 0.1952 - val_loss: 0.0242 - val_mean_squared_error: 0.0242 - lr: 0.0010
Epoch 5/50
33/33 [==============================] - 1s 19ms/step - loss: 0.1540 - mean_squared_error: 0.1540 - val_loss: 0.0173 - val_mean_squared_error: 0.0173 - lr: 0.0010
Epoch 6/50
33/33 [==============================] - 1s 17ms/step - loss: 0.1289 - mean_squared_error: 0.1289 - val_loss: 0.0150 - 

[0.0005113074439577758, 0.0005113074439577758]

In [79]:
modelbest = Sequential([
    Dense(units=250, activation='relu', input_dim=317),  # 250 nöronlu giriş katmanı
    BatchNormalization(),
    Dropout(0.2),

    Dense(units=350, activation='relu'),  # 350 nöronlu 1. gizli katman
    BatchNormalization(),
    Dropout(0.2),

    Dense(units=200, activation='relu'),  # 200 nöronlu 2. gizli katman
    BatchNormalization(),
    Dropout(0.2),

    Dense(units=1, activation='linear')  # 1 nöronlu çıktı katmanı (fiyat tahmini)
])

modelbest.compile(loss="mse", optimizer="adam", metrics=["mse"])
modelbest.fit(xtrain, ytrain, epochs=50,
                  validation_data=(xval, yval))
modelbest.evaluate(X_test, y_test)

Epoch 1/50
33/33 [==============================] - 2s 17ms/step - loss: 37996519424.0000 - mse: 37996519424.0000 - val_loss: 43016249344.0000 - val_mse: 43016249344.0000
Epoch 2/50
33/33 [==============================] - 0s 9ms/step - loss: 37994541056.0000 - mse: 37994541056.0000 - val_loss: 43015065600.0000 - val_mse: 43015065600.0000
Epoch 3/50
33/33 [==============================] - 0s 9ms/step - loss: 37992677376.0000 - mse: 37992677376.0000 - val_loss: 43014340608.0000 - val_mse: 43014340608.0000
Epoch 4/50
33/33 [==============================] - 0s 8ms/step - loss: 37990526976.0000 - mse: 37990526976.0000 - val_loss: 43013967872.0000 - val_mse: 43013967872.0000
Epoch 5/50
33/33 [==============================] - 0s 10ms/step - loss: 37987741696.0000 - mse: 37987741696.0000 - val_loss: 43011915776.0000 - val_mse: 43011915776.0000
Epoch 6/50
33/33 [==============================] - 0s 9ms/step - loss: 37984497664.0000 - mse: 37984497664.0000 - val_loss: 43013943296.0000 - val_

[39969771520.0, 39969771520.0]

In [80]:
test_y = pd.read_csv('/content/sample_submission.csv')
test_y

,Id,SalePrice
0,1461,2667.7039
1,1462,88669.0500
2,1463,2667.7039
3,1464,2667.7039
4,1465,9117.2330
...,...,...
1454,2915,9280.9170
1455,2916,10645.6490
1456,2917,2667.7039
1457,2918,2667.7039


In [56]:
predictions = modelbest.predict(sam_test)
predictions

46/46 [==============================] - 0s 4ms/step


array([[ 2667.7039],
       [88669.05  ],
       [ 2667.7039],
       ...,
       [ 2667.7039],
       [ 2667.7039],
       [ 2667.7039]], dtype=float32)

In [57]:



test_y['SalePrice']=predictions
test_y.head()


,Id,SalePrice
0,1461,2667.703857
1,1462,88669.046875
2,1463,2667.703857
3,1464,2667.703857
4,1465,9117.233398


In [60]:
test_y.to_csv('sample_submission.csv', index=False)